In [1]:
# Gerekli kütüphanelerin içe aktarılması:
from reportlab.pdfgen import canvas  # PDF içerisine metin ve grafik çizebilmek için ReportLab kütüphanesi
from reportlab.lib.units import cm  # Santimetre cinsinden ölçü birimini kullanabilmek için
from reportlab.pdfbase import pdfmetrics  # Özel (TTF) fontları tanımlamak ve kullanmak için
from reportlab.pdfbase.ttfonts import TTFont  # TTF formatındaki font dosyalarını yüklemek için
from PyPDF2 import PdfReader, PdfWriter  # PDF dosyalarını okumak, birleştirmek ve yazmak için
from pandas import read_excel  # Excel dosyalarını okuyabilmek için pandas kütüphanesi
import io  # Bellek içi dosya işlemleri yapmak için (BytesIO)
import copy  # PDF sayfalarını kopyalamak için gerekli

In [2]:
# DejaVu Sans fontunun kaydedilmesi:
font_path = r"C:/Users/esrab/PythonProject/ReportLab/dejavu-fonts-ttf-2.37/ttf/DejaVuSans.ttf"
pdfmetrics.registerFont(TTFont('DejaVu', font_path))

In [4]:
# Excel dosyasını okuyarak katılımcı listesini elde etme:
participant_list = read_excel("./certificate.xlsx", sheet_name="Sayfa1")  
# "certifika.xlsx" dosyasının "Sayfa1" isimli çalışma sayfasındaki verileri okuyoruz.
participant_list

,Name,Participant Type
0,Esra Budak,Participant
1,Zeynep Karaca,Participant
2,Mert caba,Participant
3,Elif Demir,Participant
4,Burak Aydın,Participant
5,Ayşe Şahin,Participant
6,Can Kutlu,Participant
7,Nazlı Can,Participant
8,Efe Arslan,Int Sci Com Member
9,Deniz Ergin,Int Sci Com Member


In [5]:
# Boş sertifika PDF dosyasını açma:
with open("Certificate.pdf", "rb") as file:  # Sertifikanın arka planı olarak kullanılacak PDF dosyasını okuma modunda açıyoruz
    blank_certificate = PdfReader(file)  # PDF dosyasını PdfReader ile yüklüyoruz
    num_pages = len(blank_certificate.pages)  # PDF dosyasındaki toplam sayfa sayısını alıyoruz
    print(f"Sayfa sayısı: {num_pages}")  # Sayfa sayısını konsola yazdırıyoruz

Sayfa sayısı: 1


In [6]:
# Yeni bir PDF oluşturmak için sayfa boyutunu ayarlıyoruz:
certifica_size = (29.704 * cm, 21.026 * cm)  
# Sayfa boyutunu santimetre cinsinden belirtiyoruz (ReportLab'un varsayılan birimi point olduğu için cm ile çeviriyoruz)
outfile = PdfWriter()  # Oluşturulacak PDF'yi yazmak için PdfWriter nesnesi

# Bellek içi bir dosya (BytesIO) oluşturup canvas yaratma:
packet = io.BytesIO()  # PDF içeriğini bellekte tutmak için BytesIO kullanıyoruz
canvas_obj = canvas.Canvas(packet, pagesize=certifica_size)  
# Yeni bir canvas oluşturuyoruz, sayfa boyutunu yukarıda ayarladığımız değer olarak belirtiyoruz
# Excel'deki her bir katılımcı için sertifika sayfası oluşturma:
for index, row in participant_list.iterrows():
    # Metnin çizileceği koordinatları santimetre cinsinden belirliyoruz:
    x = 10.01 * cm  
    y = 10.91 * cm  

    # Excel'den "Name" ve "Participant Type" sütunlarındaki verileri alıyoruz:
    name, participant_type = row['Name'], row['Participant Type']
    print(f"Name: {name}, Participant Type: {participant_type}")  # Bilgileri konsola yazdırıyoruz

    # Katılımcının adını PDF üzerine yazdırma:
    canvas_obj.setFont("DejaVu", 20)  
    # Daha önce kaydettiğimiz DejaVu fontunu 20 punto büyüklüğünde ayarlıyoruz
    canvas_obj.drawString(x, y, name)  
    # Belirlediğimiz (x, y) koordinatlarında katılımcının adını PDF'ye yazdırıyoruz

    # Şu anki sayfayı bitirip yeni bir sayfa başlatıyoruz:
    canvas_obj.showPage()

# Tüm çizimleri kaydediyoruz:
canvas_obj.save()  

Name: Esra Budak, Participant Type: Participant
Name: Zeynep Karaca, Participant Type: Participant
Name: Mert caba, Participant Type: Participant
Name: Elif Demir, Participant Type: Participant
Name: Burak Aydın, Participant Type: Participant
Name: Ayşe Şahin, Participant Type: Participant
Name: Can Kutlu, Participant Type: Participant
Name: Nazlı Can, Participant Type: Participant
Name: Efe Arslan, Participant Type: Int Sci Com Member
Name: Deniz Ergin, Participant Type: Int Sci Com Member


In [8]:
# Akış sıfırlanıyor ve flush işlemi yapıyoruz
packet.flush()
packet.seek(0)  # Akış başa alınacak

# PdfReader ile oluşturduğumuz PDF'yi okuyoruz
canvas_pdf = PdfReader(packet)

In [9]:
# Bellekte oluşturduğumuz PDF'yi PdfReader ile yüklüyoruz
blank_certificate = PdfReader("Certificate.pdf")  
# Sertifikanın arka plan PDF dosyasını yeniden yüklüyoruz (daha önce açılmıştı ama tekrar kullanıyoruz)
output = PdfWriter()  # Son PDF'yi oluşturmak için PdfWriter nesnesi

# Canvas üzerinde oluşturduğumuz sayfaları, boş (arka plan) sertifika ile birleştiriyoruz:
for i in range(len(canvas_pdf.pages)):
    # Arka plan PDF'sinin ilk sayfasını kopyalıyoruz (her sertifika için aynı sayfa kullanılıyor)
    page = copy.copy(blank_certificate.pages[0])
    # Canvas'da oluşturduğumuz ilgili sayfayı arka plan sayfası ile birleştiriyoruz:
    page.merge_page(canvas_pdf.pages[i])
    # Birleştirilmiş sayfayı çıktı PDF'sine ekliyoruz:
    output.add_page(page)
    print(f"{i + 1}. sertifika oluşturuldu.")  # Her bir sertifikanın oluşturulduğunu konsola yazdırıyoruz

# Son olarak, oluşturulan tüm sertifikaları içeren PDF'yi dosyaya kaydediyoruz:
with open("sertifikalar.pdf", "wb") as outfile:  
    # "sertifikalar.pdf" isimli dosyayı yazma modunda açıyoruz
    output.write(outfile)  # PdfWriter nesnesindeki içeriği dosyaya yazıyoruz

1. sertifika oluşturuldu.
2. sertifika oluşturuldu.
3. sertifika oluşturuldu.
4. sertifika oluşturuldu.
5. sertifika oluşturuldu.
6. sertifika oluşturuldu.
7. sertifika oluşturuldu.
8. sertifika oluşturuldu.
9. sertifika oluşturuldu.
10. sertifika oluşturuldu.
